# Step 1a: Customize your supply chain - defining facility template (optional)
- Facility templates are to define different types of facilities in a supply chain
- A facility template defines not only what kinds of agents it contains but also how they perform (class)
- In a facility template, you can also assign agents to different "agent_type" to group agents
- For example, the following codes define a store template of a retailer:
-- Each store can be seen as a facility with a list of "products" and "storage" capacity
-- Each product in the store has both "consumer" and "seller" behaviors to denote that it can place replenishing orders and be sold to customers.

In [ ]:
"""
    RetailerFacility: &retailer_facility             
    class: "RetailerFacility"                        
    children:
      storage:
        class: "StorageUnit"
      products:
        class: "StoreProductUnit"
        is_template: true
        config:
          agent_type: "productstore"
          consumer:
            class: "ConsumerUnit"
            config:
              agent_type: "consumerstore"
          seller:
            class: "SellerUnit"
            config:
              sale_hist_len: 4
    config:
      agent_type: "facility"
"""

# Step 1b: Instantiate all facilities in the supply chain
- Refer a facility template defined before
- Instantiate parameters of each sub-agent
- Below is an example showing how a supplier and a retailer can be instantiated.

In [ ]:
"""
    facilities:
        - name: "Supplier_001" # name of the facility
          definition_ref: "SupplierFacility"
          # sku list of this facility
          skus:
            sku1: # sku name and attributes needed for this facility
              init_stock: 1000
              product_unit_cost: 90
              production_rate: 1000
              type: "production"
              cost: 90
              price: 100
              vlt: 2
           sku2:
              init_stock: 1000
              product_unit_cost: 150
              production_rate: 1000
              type: "production"
              cost: 150
              price: 200
              vlt: 3
         children:
            storage: 
            distribution:
          # config of this facility
          config:
            delay_order_penalty: 100
            order_cost: 10
        
    - name: "Retailer_001"
          definition_ref: "RetailerFacility"
          skus:
            sku1:
              price: 200
              cost: 100
              init_stock: 40
              sale_gamma: 20
              backlog_ratio: 0.1
              vlt: 1
            sku2:
              price: 350
              cost: 200
              init_stock: 160
              sale_gamma: 80
              backlog_ratio: 0.1
              vlt: 1
         children:
            storage: *small_storage
"""




# In case we want to define a retailer store with demands of SKUs are sampled from historical data, this can be simply done by changing the "class" of seller, see below for instance.

"""
    RetailerFacility: &raw_data_retailer_facility
        class: "OuterRetailerFacility"
        children:
          storage:
            class: "StorageUnit"
          products:
            class: "StoreProductUnit"
            is_template: true
            config:
              agent_type: "consumer"
              consumer:
                class: "ConsumerUnit"
              seller:
                class: "RawDataSellerUnit"    ######################
                config:
                  sale_hist_len: 4
        config:
          agent_type: "facility"
          seller_sampler_type: data
          sku_column: "SKU"
          price_column: "Price"
          sale_column: "Sales"
          datetime_column: "DT"
          file_path: "/path/to/data.csv"
      
"""

# Step 2: Defining fulfillment relation

In [ ]:
"""
      topology:
        Warehouse_001:
          sku1:
            - "Supplier_001"
          sku2:
            - "Supplier_001"
          sku3:
            - "Supplier_001"
        Retailer_001:
          sku1:
            - "Warehouse_001"
          sku2:
            - "Warehouse_001"
          sku3:
            - "Warehouse_001"
"""

# Step 3: Defining inventory policies

In [16]:
import sys
from os.path import dirname, realpath
import numpy as np
import torch

from maro.rl import (
    DQN, DQNConfig, EpisodeBasedSchedule, FullyConnectedBlock, NullPolicy, OptimOption, QNetForDiscreteActionSpace,
    StepBasedSchedule, UniformSampler
)

from or_policy.minmax_policy import ConsumerMinMaxPolicy
from or_policy.eoq_policy import ConsumerEOQPolicy as ProducerEOQPolicy
from or_policy.base_policy import ProducerBaselinePolicy

sc_code_dir = dirname(realpath(__file__))
sys.path.insert(0, sc_code_dir)
from config import config

def get_dqn_consumer_policy(config):
    q_net = SimpleQNet(
        FullyConnectedBlock(**cfg["model"]["network"]),
        optim_option=OptimOption(**cfg["model"]["optimization"]),
        device=cfg["model"]["device"]
    )
    experience_manager = UniformSampler(**cfg["experience_manager"])
    return DQN(q_net, experience_manager, DQNConfig(**cfg["algorithm_config"]))

def get_eoq_producer_policy(config):
    return ProducerEOQPolicy(config)

ModuleNotFoundError: No module named 'maro'

# Step 4: Mapping agents to policies

In [ ]:
config = {}
policy_dict = {
    'consumer': get_dqn_consumer_policy(config),
    'producer': get_eoq_producer_policy(config)
}

# Step 5: Training and evaluation

In [15]:
!bash run.sh

/bin/bash: run.sh: No such file or directory
